In [1]:
from itertools import combinations
import random
random.seed(42)

import torch
from torch import nn
import torch.nn.functional as F

In [2]:
nums = list(combinations(range(1, 21), 2))
random.shuffle(nums)
num_size = 10

# targets = torch.tensor([sum(num) for num in nums])
# targets.shape
len(nums)

190

In [3]:
# hyperparameters
batch_size = 4	# how many independent processes will we process in parallel
block_size = 8	# whar is the maximum context length for prediction
max_iters = 5000
eval_intervals = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 6
n_layer = 6
dropout = 0.2


In [4]:
text = list('0123456789+=$')
itos = dict(enumerate(text))
stoi = {s: i for i, s in enumerate(text)}

In [5]:
decode = lambda arr: ''.join(itos[char] for char in arr if itos[char] != '$')
e = lambda s: list(stoi[char] for char in s)

def encode(arr: list[int]):
    # arr.append(sum(arr))
    n = block_size - len(arr) - sum(len(str(num)) for num in arr)
    if n < 0:
        print('ERROR')
        print(arr, n)

    out = e('$')*n
    for char in arr:
        out += e(str(char) + '+')
    out = out[:-1] + e('=')
    # out = out[:-1] + e('=' + str(arr[-1]))

    return (out, e(str(sum(arr))))

# def encode(arr: list[int]):
#     arr = list(arr)
#     arr.append(sum(arr))
#     n = (block_size - len(arr) - sum(len(str(num)) for num in arr) ) // len(arr)

#     out = []
#     [out.extend(arr) for arr in [[stoi['0']]*n + [stoi[i] for i in str(num)] + [stoi['+']] for num in arr[:-1]]]
#     out.pop(-1)
#     out.extend([stoi['=']] + (block_size - len(out) - 1 - len(str(arr[-1])))*[stoi['0']] + [stoi[i] for i in str(arr[-1])])

#     return out

# def encode(s: str):
#     arr = s.split('+')
#     arr += arr.pop(-1).split('=')
#     n = (block_size - len(s)) // len(arr)
#     out = ''.join('0'*n + num + '+' for num in arr[:-1])
#     out = out[:-1] + '=' + (block_size - len(out) - len(arr[-1]))*'0' + arr[-1]
#     out = [stoi[i] for i in out]
#     return out

# encode = lambda s: [stoi[char] for char in s]

In [6]:
print(encode([10, 11]))
print([decode(i) for i in encode([10, 11])])

([12, 12, 1, 0, 10, 1, 1, 11], [2, 1])
['10+11=', '21']


In [7]:
# train test split
X = []
Y = []
for num in nums:
    arr, res = encode(num)
    for y in res:
        X.append(arr)
        Y.append(y)
        arr = arr[1:] + [y]

X = torch.tensor(X)
Y = torch.tensor(Y)

print(X[:5])
print(Y[:5])

n = int(0.9 * len(nums))
train_X = X[:n]
train_Y = Y[:n]
val_X = X[n:]
val_Y = Y[n:]

tensor([[12, 12, 12, 12,  1, 10,  4, 11],
        [12, 12, 12,  4, 10,  1,  5, 11],
        [12, 12,  4, 10,  1,  5, 11,  1],
        [12, 12, 12,  1, 10,  1,  4, 11],
        [12, 12,  1, 10,  1,  4, 11,  1]])
tensor([5, 1, 9, 1, 5])


In [8]:
torch.manual_seed(1337)
def get_batch(split):
    data = [train_X, train_Y] if split == "train" else [val_X, val_Y]
    ix = torch.randint(len(data[0]), (batch_size, ))
    print(ix)

    x = torch.stack([data[0][i] for i in ix])
    y = torch.stack([data[1][i] for i in ix])
    #x = torch.stack([data[i:i+block_size] for i in ix])
    #y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)
    return x, y

print(get_batch("train"))


tensor([163, 112, 104, 106])
(tensor([[12, 12,  5, 10,  1,  9, 11,  2],
        [12, 12, 12,  3, 10,  1,  0, 11],
        [12,  1,  1, 10,  2,  0, 11,  3],
        [12,  1,  1, 10,  1,  5, 11,  2]]), tensor([4, 1, 1, 6]))


# New Data System

In [22]:
res_number = 2
nums = list(combinations(range(1, 21), 2))
random.shuffle(nums)

n = int(0.9*len(nums))
train_nums = torch.tensor(nums[:n], dtype=torch.long, device=device)
val_nums = torch.tensor(nums[n:], dtype=torch.long, device=device)

num_size = 16
len(nums)

190

In [21]:
train_nums.shape, val_nums.shape

(torch.Size([171, 2]), torch.Size([19, 2]))

In [46]:
# decode = lambda arr: ''.join(itos[char] for char in arr if itos[char] != '$')
decode = lambda arr: ''.join(itos[char] for char in arr)
e = lambda s: list(stoi[char] for char in s)

def encode(arr: list[int], r: int, tensor: bool =False) -> tuple[list[int], list[int]]:
    if tensor: arr = arr.tolist()
    # print(f'{block_size=}\n{len(arr)=}\n{sum(len(str(num)) for num in arr)=}')
    n = block_size - len(arr) - sum(len(str(num)) for num in arr)
    if n < 0:
        print('ERROR')
        print(arr, n)

    out1 = e('$')*n
    for char in arr:
        out1 += e(str(char) + '+')
    out1 = out1[:-1] + e('=')

    summ = str(sum(arr))
    out2 = e('0'*(res_number - len(summ)) + summ)

    out = out1 + out2
    out1 = out[r:][: block_size]
    out2 = out[r+1:][: block_size]

    if tensor:
        return torch.tensor((out1, out2), dtype=torch.long, device=device)
    return (out1, out2)
print(e('$$10+11=21'))
print(encode([10, 11], 1))
print(encode(torch.tensor([10, 11]), 1, True))


[12, 12, 1, 0, 10, 1, 1, 11, 2, 1]
([12, 1, 0, 10, 1, 1, 11, 2], [1, 0, 10, 1, 1, 11, 2, 1])
tensor([[12,  1,  0, 10,  1,  1, 11,  2],
        [ 1,  0, 10,  1,  1, 11,  2,  1]])


In [47]:
arr = list(range(20))
print(arr)
arr[:5]

t = torch.tensor([5, 4])
t.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


[5, 4]

In [57]:
torch.manual_seed(1337)
def get_batch(split: str):
    data = train_nums if split == "train" else val_nums
    ix = torch.randint(len(data) * res_number, (4, ))
    print(ix)

    x = torch.stack([encode(data[i//res_number], i%res_number, True) for i in ix])
    # y = torch.stack([data[1][i] for i in ix])

    # x, y = x.to(device), y.to(device)
    # return x, y
    return x

print(get_batch("train"))


tensor([163, 283, 104, 106])
tensor([[[12, 12,  8, 10,  1,  3, 11,  2],
         [12,  8, 10,  1,  3, 11,  2,  1]],

        [[12, 12,  2, 10,  1,  8, 11,  2],
         [12,  2, 10,  1,  8, 11,  2,  0]],

        [[12, 12, 12, 12,  1, 10,  3, 11],
         [12, 12, 12,  1, 10,  3, 11,  0]],

        [[12, 12, 12,  7, 10,  1,  4, 11],
         [12, 12,  7, 10,  1,  4, 11,  2]]])


In [56]:
t = get_batch("train")
print(t)
X = t[:, 0, :]
Y = t[:, 1, :]
X, Y

tensor([233, 162,  44, 324])
tensor([[[12, 12,  4, 10,  2,  0, 11,  2],
         [12,  4, 10,  2,  0, 11,  2,  4]],

        [[12, 12, 12,  8, 10,  1,  3, 11],
         [12, 12,  8, 10,  1,  3, 11,  2]],

        [[12, 12,  1,  4, 10,  1,  7, 11],
         [12,  1,  4, 10,  1,  7, 11,  3]],

        [[12, 12, 12,  6, 10,  1,  4, 11],
         [12, 12,  6, 10,  1,  4, 11,  2]]])


(tensor([[12, 12,  4, 10,  2,  0, 11,  2],
         [12, 12, 12,  8, 10,  1,  3, 11],
         [12, 12,  1,  4, 10,  1,  7, 11],
         [12, 12, 12,  6, 10,  1,  4, 11]]),
 tensor([[12,  4, 10,  2,  0, 11,  2,  4],
         [12, 12,  8, 10,  1,  3, 11,  2],
         [12,  1,  4, 10,  1,  7, 11,  3],
         [12, 12,  6, 10,  1,  4, 11,  2]]))

In [53]:
X.shape

torch.Size([4, 2, 8])